# Experiment 02 - Data

- [x] mask entries in larger dataset in long-format
- [x] mask peptides based on their frequency in samples (probability of being observed)
- [ ] create training data set without masked values for each model
    - Denoising AE
    - FNN based on embeddings (Collaborative Filtering)
    - VAE
- [ ] restrict to only a training data split of consective data: Increase number of samples.
    - focus on best reconstruction performance
    - mean comparison
    - 

In [ ]:
from pprint import pprint
from src.nb_imports import *


import vaep.io_images
import seaborn

from pathlib import Path
from src import metadata


import logging
from src.logging import setup_logger

logger = setup_logger(logger=logging.getLogger('vaep'))
logger.info("Experiment 02")

figures = {}  # collection of ax or figures

ADD_TENSORBOARD = False

In [ ]:
# None takes all
N_SAMPLES = None

Change some Matplotlib configuration defaults

In [ ]:
plt.rcParams.update({'xtick.labelsize': 'xx-large',
                     'ytick.labelsize': 'xx-large'})

## Raw data

In [ ]:
# only some sample have many missings

FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / 'df_intensities_N07813_M01000'  # all
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / 'df_intensities_N_07637_M01000'  # 60%
FN_PEPTIDE_INTENSITIES = config.FOLDER_DATA / 'df_intensities_N_07285_M01000'  # 90%

In [ ]:
analysis = AnalyzePeptides(fname=FN_PEPTIDE_INTENSITIES, nrows=None)
analysis.df.columns.name = 'peptide'
analysis.log_transform(np.log2)
analysis

In [ ]:
# some date are not possible in the indices
rename_indices_w_wrong_dates = {'20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_03': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_03',
                                '20180230_QE10_nLC0_MR_QC_MNT_Hela_12': '20180330_QE10_nLC0_MR_QC_MNT_Hela_12',
                                '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_01': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_01',
                                '20180230_QE10_nLC0_MR_QC_MNT_Hela_11': '20180330_QE10_nLC0_MR_QC_MNT_Hela_11',
                                '20161131_LUMOS1_nLC13_AH_MNT_HeLa_long_02': '20161130_LUMOS1_nLC13_AH_MNT_HeLa_long_02'}
analysis.df.rename(index=rename_indices_w_wrong_dates, inplace=True)

### Select N consecutive samples

In [ ]:
# sort index
analysis.df.sort_index(inplace=True)
analysis.df_all = analysis.df

In [ ]:
N_SAMPLES = min(len(analysis.df), N_SAMPLES) if N_SAMPLES else len(analysis.df)

import random
random.seed(42)
def get_consecutive_data_indices(df, n_samples=N_SAMPLES):
    index = df.sort_index().index
    start_sample = len(index) - n_samples
    start_sample = random.randint(0, start_sample)
    return df.loc[index[start_sample:start_sample+n_samples]]

_attr_name = f'df_{N_SAMPLES}'
setattr(analysis,_attr_name,get_consecutive_data_indices(analysis.df_all) )
print("Training data stored under:", _attr_name)
analysis.df = getattr(analysis, _attr_name)
analysis.df

In [ ]:
assert not analysis.df._is_view

- biological stock differences in PCA plot. Show differences in models. Only see biological variance

In [ ]:
d_meta = metadata.get_metadata_from_filenames(analysis.df.index)
analysis.df_meta = pd.DataFrame.from_dict(
    d_meta, orient='index')
analysis.df_meta

Use to find date parsing errors, used for renaming above.

In [ ]:
# invalid_dates = pd.to_datetime(analysis.df_meta.date, errors='coerce').isna()
# display(analysis.df_meta.loc[invalid_dates])
# {i : i for i in analysis.df_meta.loc[invalid_dates].index} # to rename

In [ ]:
analysis.df_meta.describe()

See rare instrument types (potential labeling errors)

In [ ]:
N_MIN_INSTRUMENT = 10
ms_instruments = analysis.df_meta.ms_instrument.value_counts()
ms_instruments = ms_instruments[ms_instruments > N_MIN_INSTRUMENT].index
mask = ~analysis.df_meta.ms_instrument.isin(ms_instruments)
analysis.df_meta.loc[mask]

## PCA plot of raw data

In [ ]:
import itertools
from sklearn.impute import SimpleImputer
X = SimpleImputer().fit_transform(analysis.df)
X = vaep.pandas._add_indices(X, analysis.df)
assert X.isna().sum().sum() == 0

pca = analyzers.run_pca(X)
cols = list(pca.columns)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(
    15, 20), constrained_layout=True)

from collections import namedtuple
Dim = namedtuple('DimensionsData', 'N M')
analysis.dim = Dim(*analysis.df.shape)

fig.suptitle(f'First two Principal Components of {analysis.dim.M} most abundant peptides \n for {analysis.dim.N} samples', fontsize=30)


# by instrument
ax = axes[0]
pca['ms_instrument'] = analysis.df_meta['ms_instrument'].astype('category')
# for name, group in pca.groupby('ms_instrument'):
#     ax.scatter(x=group[cols[0]], y=group[cols[1]], label=name)
seaborn.scatterplot(x=pca[cols[0]], y=pca[cols[1]], hue=pca['ms_instrument'], ax=ax, palette='deep')
ax.set_title('by category', fontsize=18)
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))

# by dates
ax = axes[1]
ax.set_title('by date', fontsize=18)
path_collection = analyzers.scatter_plot_w_dates(
    ax, pca, dates=analysis.df_meta.date, errors='raise')
path_collection = analyzers.add_date_colorbar(path_collection, fig)

In [ ]:
vaep.io_images._savefig(fig, config.FIGUREFOLDER/ f'pca_plot_raw_data_{analysis.fname_stub}')

### Single plots without titles

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(
    15, 10), constrained_layout=True)
seaborn.scatterplot(x=pca[cols[0]], y=pca[cols[1]], hue=pca['ms_instrument'], ax=ax, palette='deep')
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))

vaep.io_images._savefig(fig, config.FIGUREFOLDER/ f'pca_plot_raw_data_{analysis.fname_stub}_by_category')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(
    15, 10), constrained_layout=True)


path_collection = analyzers.scatter_plot_w_dates(
    ax, pca, dates=analysis.df_meta.date, errors='raise')
path_collection = analyzers.add_date_colorbar(path_collection, fig)

vaep.io_images._savefig(fig, config.FIGUREFOLDER/ f'pca_plot_raw_data_{analysis.fname_stub}_by_date')

## Interactive PCA plots of raw data

In [ ]:
import plotly.express as px
# seaborn.scatterplot(x=pca[cols[0]], y=pca[cols[1]], hue=pca['ms_instrument'], ax=ax, palette='deep')
fig = px.scatter(pca, x=cols[0], y=cols[1], color="ms_instrument")
fig.show()

## Long format

- Data in long format: (peptide, sample_id, intensity)
- no missing values kept
- 

In [ ]:
def get_long_format(self, colname_values='intensity', inplace=False):
    df = self.df
    df_long = df.unstack().dropna().to_frame(colname_values)
    df_long = df_long.reset_index('Sample ID')
    if inplace:
        self.df_long = df_long
        return
    return df_long


get_long_format(analysis, inplace=True)
analysis.df_long.head()

In [ ]:
assert analysis.df_long.isna().sum().sum() == 0, "There are still missing values in the long format."

In [ ]:
def get_wide_format(self, columns='Sample ID', name_values='intensity', inplace=False):
    df_wide = self.df_long.pivot(columns=columns, values=name_values)
    df_wide = df_wide.T
    if inplace:
        self.df_wide = df_wide
        return
    return df_wide


get_wide_format(analysis, inplace=True)
analysis.df_wide.head()

In [ ]:
assert analysis.df_wide.isna().sum().sum() > 0, "There are no missing values left in the wide format"

### Sampling peptides by their frequency (important for later)

- higher count, higher probability to be sampled into training data
- missing peptides are sampled both into training as well as into validation dataset
- everything not in training data is validation data

In [ ]:
# freq_per_peptide = analysis.df.unstack().to_frame('intensity').reset_index(1, drop=True)
freq_per_peptide = analysis.df_long['intensity']
freq_per_peptide = freq_per_peptide.notna().groupby(level=0).sum()

In [ ]:
# df_long = analysis.df.unstack().to_frame('intensity').reset_index(1)
analysis.df_train = analysis.df_long.groupby(
    by='Sample ID').sample(frac=0.95, weights=freq_per_peptide, random_state=42)
analysis.df_train = analysis.df_train.reset_index().set_index([
    'Sample ID', 'peptide'])
analysis.df_train

## MultiIndex 

- use mulitindex for obtaining validation split

In [ ]:
analysis.df_long = analysis.df_long.reset_index(
).set_index(['Sample ID', 'peptide'])
analysis.df_long.head()

In [ ]:
analysis.indices_valid = analysis.df_long.index.difference(
    analysis.df_train.index)
analysis.df_valid = analysis.df_long.loc[analysis.indices_valid]

In [ ]:
assert len(analysis.df_long) == len(analysis.df_train) + len(analysis.df_valid)